In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
sentiment_df = pd.read_csv("/content/fear_greed_index.csv")
trader_df = pd.read_csv("/content/historical_data.csv")

# Preprocess sentiment data
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df = sentiment_df[['date', 'classification']]

# Preprocess trader data
trader_df['Timestamp IST'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trader_df['date'] = trader_df['Timestamp IST'].dt.date
trader_df['date'] = pd.to_datetime(trader_df['date'])
trader_df['Closed PnL'] = pd.to_numeric(trader_df['Closed PnL'], errors='coerce')

# Merge datasets on date
merged_df = pd.merge(trader_df, sentiment_df, on='date', how='left')

# Map sentiment to numeric scores
sentiment_order = {'Extreme Fear': 0, 'Fear': 1, 'Neutral': 2, 'Greed': 3, 'Extreme Greed': 4}
merged_df['sentiment_score'] = merged_df['classification'].map(sentiment_order)

# ANALYSIS

# 1. Performance by sentiment and trade side
performance_by_sentiment_side = merged_df.groupby(['classification', 'Side'])['Closed PnL'].agg(['mean', 'median', 'std', 'count']).reset_index()
print("Performance by Sentiment and Side:\n", performance_by_sentiment_side)

# 2. Sentiment score correlation with average PnL
sentiment_score_pnl = merged_df.groupby('sentiment_score')['Closed PnL'].mean().reset_index()
print("\nAverage PnL by Sentiment Score:\n", sentiment_score_pnl)

# 3. Top 3 performing accounts under each sentiment
top_accounts_by_sentiment = (
    merged_df.groupby(['classification', 'Account'])['Closed PnL']
    .sum()
    .reset_index()
    .sort_values(['classification', 'Closed PnL'], ascending=[True, False])
)
top_3_per_sentiment = top_accounts_by_sentiment.groupby('classification').head(3)
print("\nTop 3 Performing Accounts by Sentiment:\n", top_3_per_sentiment)

#VISUALIZATIONS

# Boxplot of PnL distribution by sentiment
plt.figure(figsize=(10, 6))
sns.boxplot(x='classification', y='Closed PnL', data=merged_df, showfliers=False, palette='coolwarm')
plt.title('PnL Distribution by Market Sentiment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Line plot for sentiment score vs average PnL
plt.figure(figsize=(8, 5))
sns.lineplot(data=sentiment_score_pnl, x='sentiment_score', y='Closed PnL', marker='o')
plt.title('Average PnL vs Sentiment Score')
plt.xlabel('Sentiment Score (0=Extreme Fear, 4=Extreme Greed)')
plt.ylabel('Average Closed PnL')
plt.grid(True)
plt.tight_layout()
plt.show()
